## Books Recommender System using Clustering 

In [1]:
%pip install pandas numpy matplotlib seaborn scikit-learn 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
books = pd.read_csv('data/books.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
users = pd.read_csv('data/users.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
ratings = pd.read_csv('data/ratings.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1') 


C:\Users\diego\AppData\Local\Temp\ipykernel_12912\1850883518.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('data/books.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields

In [4]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [5]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [6]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "autor",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"}, inplace = True)


# Lets remane some wierd columns name
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

# Lets remane some wierd columns name
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [7]:
print(books.shape, users.shape, ratings.shape, sep='\n')

(271360, 6)
(278858, 3)
(1149780, 3)


In [8]:
ratings['user_id'].value_counts().shape

(105283,)

In [9]:
ratings['user_id'].unique().shape

(105283,)

In [10]:
# Lets store users who had at least rated more than 200 books
#x = ratings['user_id'].value_counts() > 200
x = ratings['user_id'].value_counts().between(50, 1000, inclusive=True)

C:\Users\diego\AppData\Local\Temp\ipykernel_12912\2458003562.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  x = ratings['user_id'].value_counts().between(50, 1000, inclusive=True)


In [11]:
x[x].shape

(3310,)

In [12]:
y= x[x].index

In [13]:
y

Int64Index([129074, 252695, 197659, 211426, 265313, 187145,  56856, 127200,
            235842, 131046,
            ...
             14374, 270801, 141157, 154844, 214717,  22409, 265784, 261685,
            103630, 144478],
           dtype='int64', length=3310)

In [14]:
ratings = ratings[ratings['user_id'].isin(y)]

In [15]:
ratings.head()

,user_id,ISBN,rating
173,276847,0446364193,0
174,276847,3257200552,5
175,276847,3379015180,0
176,276847,3404145909,8
177,276847,3404148576,8


In [16]:
ratings.shape

(547099, 3)

In [17]:
# Now join ratings with books

ratings_with_books = ratings.merge(books, on='ISBN')

In [18]:
ratings_with_books.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...
1,5483,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...
2,8362,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...
3,9177,0446364193,7,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...
4,11400,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...


In [19]:
ratings_with_books.shape

(495386, 8)

In [20]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [21]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,3
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Clifford Visita El Hospital (Clifford El Gran...,1
3,Deceived,1
4,Earth Prayers From around the World: 365 Pray...,5


In [22]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [23]:
number_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,3
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Clifford Visita El Hospital (Clifford El Gran...,1
3,Deceived,1
4,Earth Prayers From around the World: 365 Pray...,5


In [24]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [25]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
1,5483,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
2,8362,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
3,9177,0446364193,7,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
4,11400,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158


In [26]:
final_rating.shape

(495386, 9)

In [27]:
# Lets take those books which got at least 50 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 20]

In [28]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
1,5483,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
2,8362,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
3,9177,0446364193,7,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
4,11400,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158


In [29]:
final_rating.shape

(156754, 9)

In [30]:
# lets drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [31]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,276847,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
1,5483,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
2,8362,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
3,9177,0446364193,7,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158
4,11400,0446364193,0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,http://images.amazon.com/images/P/0446364193.0...,158


In [32]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [33]:
book_pivot

user_id,183,243,254,507,626,638,643,741,882,929,...,277639,277928,277965,278026,278137,278144,278188,278582,278633,278843
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 Lb. Penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16 Lighthouse Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1984,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"\O\"" Is for Outlaw""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
book_pivot.shape

(3260, 3221)

In [35]:
book_pivot.fillna(0, inplace=True)

In [36]:
book_pivot

user_id,183,243,254,507,626,638,643,741,882,929,...,277639,277928,277965,278026,278137,278144,278188,278582,278633,278843
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Entrenamiento del Modelo

In [37]:
from scipy.sparse import csr_matrix

In [38]:
book_sparse = csr_matrix(book_pivot)

In [39]:
book_sparse

<3260x3221 sparse matrix of type '<class 'numpy.float64'>'
	with 50831 stored elements in Compressed Sparse Row format>

In [40]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [41]:
# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [42]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [43]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors = 6)

In [44]:
distance

array([[ 0.        , 24.75883681, 26.92582404, 28.33725463, 28.61817604,
        28.77498914]])

In [45]:
suggestion

array([[ 237,  225, 2678, 2628,  845,  135]], dtype=int64)

In [46]:
for i in range(len(suggestion)):
    print(book_pivot. index[suggestion[i]])


Index(['Assassins: Assignment: Jerusalem, Target: Antichrist (Left Behind #6)',
       'Apollyon: The Destroyer Is Unleashed (Left Behind #5)',
       'The Mark: The Beast Rules the World (Left Behind #8)',
       'The Least Likely Bride', 'Federation (Star Trek)', 'Acts of Love'],
      dtype='object', name='title')


In [47]:
book_pivot.index[3]

'1984'

In [48]:
books_name = book_pivot.index  #guardo los nombres de libros

In [49]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [50]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064864.01.LZZZZZZZ.jpg'

In [51]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [52]:
book_name[0]

Index(['Assassins: Assignment: Jerusalem, Target: Antichrist (Left Behind #6)',
       'Apollyon: The Destroyer Is Unleashed (Left Behind #5)',
       'The Mark: The Beast Rules the World (Left Behind #8)',
       'The Least Likely Bride', 'Federation (Star Trek)', 'Acts of Love'],
      dtype='object', name='title')

In [53]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [54]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

http://images.amazon.com/images/P/084232920X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0842329161.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0842332251.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/055358068X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0671894226.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/080411787X.01.LZZZZZZZ.jpg


In [55]:
import pickle 
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(books_name, open('artifacts/book_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [56]:
## Serialización con Joblib
#import joblib
#joblib.dump(model, 'artifacts/model.joblib')
#joblib.dump(books_name, 'artifacts/books_name.joblib')
#joblib.dump(final_rating, 'artifacts/final_rating.joblib')
#joblib.dump(book_pivot, 'artifacts/book_pivot.joblib')

In [57]:
#test no necesario
#np.where(book_pivot.index == "2nd Chance")[0][0]

In [58]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors = 6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [65]:
def recommend_book1(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)
    
    recommend_list = []
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            recommend_list.append(j)
    
    return recommend_list

In [67]:
book_name = "You Belong To Me"
recommend_list = recommend_book1(book_name)

In [70]:
recommend_list

['You Belong To Me',
 'The Anastasia Syndrome',
 'Mask',
 'Raven (Orphans)',
 'Scruples Two',
 'While My Pretty One Sleeps']

In [73]:
formatted_list = "\n".join(["* " + book for book in recommend_list])
Text = f"Estas son algunas referencias que las personas recomiendan:\n\n{formatted_list}"
print(Text)

Estas son algunas referencias que las personas recomiendan:

* You Belong To Me
* The Anastasia Syndrome
* Mask
* Raven (Orphans)
* Scruples Two
* While My Pretty One Sleeps


In [76]:
Text1 = f"Gracias por tu review!\nTe obsequiamos un bono de 15% para tu proximo libro\nEstas son algunas referencias que las personas recomiendan:\n\n{formatted_list}"
print(Text1)

Gracias por tu review!
Te obsequiamos un bono de 15% para tu proximo libro
Estas son algunas referencias que las personas recomiendan:

* You Belong To Me
* The Anastasia Syndrome
* Mask
* Raven (Orphans)
* Scruples Two
* While My Pretty One Sleeps


In [113]:
!python --version


Python 3.9.12


In [ ]:
#https://www.youtube.com/watch?v=cGoAhtl-5jA&list=PLkz_y24mlSJa37r2xNDyEgt0Z4ilHtJ07&index=13&ab_channel=DSwithBappy